In [6]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [7]:
%%sql
drop table if exists Budgeted, Actual;
create table Budgeted (
    task integer not null primary key,
    category integer not null,
    est_cost decimal(8,2) not null
);

create table actual (
    voucher integer not null primary key,
    task integer not null references Budgeted(task),
    act_cost decimal(8,2) not null
);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
Done.


[]

In [8]:
%%sql
INSERT INTO Budgeted VALUES(1, 9100, 100.00);
INSERT INTO Budgeted VALUES(2, 9100,  15.00);
INSERT INTO Budgeted VALUES(3, 9100,   6.00);
INSERT INTO Budgeted VALUES(4, 9200,   8.00);
INSERT INTO Budgeted VALUES(5, 9200,  11.00);

INSERT INTO Actual VALUES(1,  1,  10.00);
INSERT INTO Actual VALUES(2,  1,  20.00);
INSERT INTO Actual VALUES(3,  1,  15.00);
INSERT INTO Actual VALUES(4,  2,  32.00);
INSERT INTO Actual VALUES(5,  4,  8.00);
INSERT INTO Actual VALUES(6,  5,  3.00);
INSERT INTO Actual VALUES(7,  5,  4.00);

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

## やりたいこと
+ 各categoryについて、budgetedとactualを出す
  + join(または相関サブクエリ)とgroupが必要。相関サブクエリを使用する場合はどうかけるか?

In [14]:
%%sql
select category, sum(est_cost), sum(sum_act)
from Budgeted as b
left join (
    select task, sum(act_cost)
    from Actual
    group by task
) as a(task, sum_act)
on b.task = a.task
group by category

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


category,sum,sum_1
9100,121.00,77.00
9200,19.00,15.00


In [28]:
%%sql
select category, sum(b.est_cost) as est,
    sum((
        select sum(act_cost)
        from Actual as a
        where b.task = a.task
    )) as act
from Budgeted as b
group by category

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


category,sum,act
9100,121.00,77.00
9200,19.00,15.00


In [30]:
%%sql
select category, sum(b.est_cost) as est,
    (
        select sum(a.act_cost)
        from actual as a
        where a.task between min(b.task) and max(b.task)
    ) as act
from Budgeted as b
group by category

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


category,est,act
9100,121.00,77.00
9200,19.00,15.00
